In [4]:
import win32com.client as win32
import clipboard
import datetime
import os
import re
import pandas
import pandas as pd


threatType = []
threatAddr = []
threatURL = []
threatUrlName = []
hwpList = []

path = "C:/Temp/"
# path = "D:/.processing/"

def main():
    
    fileList = os.listdir(path)
    (threatAddr,threatType,threatURL,threatUrlName) = contentExport(fileList)

    dfIPaddr = pd.DataFrame(
        {
            "Type" : threatType,
            "IP ADDR": threatAddr
        }
    )

    dfURL = pd.DataFrame(
        {
            "URL NAME" : threatUrlName,
            "URL" : threatURL
        }
    )

    dfIPaddr.to_csv(path + "threatIPADDR.csv",index=False, encoding="utf-8-sig")
    dfURL.to_csv(path + "threatURL.csv",index=False, encoding="utf-8-sig")

def contentExport(fileList):
    for index in fileList:
        hwp = win32.gencache.EnsureDispatch("HWPFrame.HwpObject")
        hwp.RegisterModule("FilePathCheckDLL", "FilePathCheckerModule")

        hwp.Open(path + index)
        ctrlCode = hwp.HeadCtrl


# 필요 데이터 추출 시작
        try:
            while ctrlCode != None:
                if ctrlCode.CtrlID == "tbl":
                    hwp.SetPosBySet(ctrlCode.GetAnchorPos(0))
                    hwp.FindCtrl()
                    hwp.Run("ShapeObjTableSelCell")
                    hwp.Run("Copy")
                    colText = clipboard.paste()
                    colText = colText.replace('\r\n', '').replace('(', ' ').replace(')', ' ')

                    if colText == "위협유형":
                        hwp.Run("TableRightCell")
                        hwp.Run("TableRightCell")
                        hwp.Run("Copy")
                        addrColText = clipboard.paste()
                        addrColText = addrColText.replace('\r\n', '')

            # IP 주소 리스트 생성
                        if addrColText == "유해 IP주소":
                            hwp.Run("TableLowerCell")
                            hwp.Run("TableCellBlockExtend")
                            posInfo = hwp.KeyIndicator()
                            curPos = posInfo[8][1:3]
                            prevPos = ''

                            while curPos !=prevPos:
                                hwp.Run("TableLowerCell")
                                prevPos = curPos
                                posInfo = hwp.KeyIndicator()
                                curPos = posInfo[8][1:3]

                            hwp.Run("Copy")
                            addrData = clipboard.paste()
                            print(addrData)
                            addrData = addrData[2:-2]


                            addrData = re.findall(r'\b(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\b', str(addrData))

                            print(index)
                            print(addrData)

                            for i in range(len(addrData)):
                                    buf = addrData[i]
                                    threatAddr.append(buf)

            # API가 셀 이동을 못해서 주소 셀을 복사 못했을 경우 루프진입
                            while True:
                                if len(addrData) == 0:

                                    hwp.Run("Close")
                                    hwp.FindCtrl()
                                    hwp.Run("ShapeObjTableSelCell")
                                    hwp.Run("TableRightCell")
                                    hwp.Run("TableRightCell")
                                    hwp.Run("TableLowerCell")
                                    hwp.Run("TableCellBlockExtend")
                                    posInfo = hwp.KeyIndicator()
                                    curPos = posInfo[8][1:3]
                                    prevPos = ''

                                    while curPos !=prevPos:
                                        hwp.Run("TableLowerCell")
                                        prevPos = curPos
                                        posInfo = hwp.KeyIndicator()
                                        curPos = posInfo[8][1:3]

                                    hwp.Run("Copy")
                                    addrData = clipboard.paste()
                                    print(addrData)
                                    addrData = addrData[2:-2]


                                    addrData = re.findall(r'\b(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\b', str(addrData))

                                    print(index)
                                    print(addrData)

                                    for i in range(len(addrData)):
                                            buf = addrData[i]
                                            threatAddr.append(buf)
                                else:
                                    break

            # 유해 IP 탐지명 리스트 생성
                            hwp.Run("Close")
                            hwp.FindCtrl()
                            hwp.Run("ShapeObjTableSelCell")
                            hwp.Run("TableRightCell")
                            hwp.Run("TableLowerCell")
                            hwp.Run("TableCellBlockExtend")

                            posInfo = hwp.KeyIndicator()
                            curPos = posInfo[8][1:3]
                            prevPos = ''

                            while curPos !=prevPos:
                                hwp.Run("TableLowerCell")
                                prevPos = curPos
                                posInfo = hwp.KeyIndicator()
                                curPos = posInfo[8][1:3]

                            hwp.Run("Copy")
                            typeData = clipboard.paste()
                            typeData = typeData[2:-2]
                            typeData = typeData.split("\r\n")

                            print(index)
                            print(typeData)

                            dataCount = len(addrData)
                            
                            print("======== TYPE NAME LOGIC ========== length %d %d" %(dataCount,len(typeData)))

                            if dataCount == len(typeData):
                                for i in range(len(typeData)):
                                    buf = typeData[i]
                                    buf = buf.replace('\r\n', '')
                                    threatType.append(buf)

                            elif dataCount < len(typeData):
                                typeData = re.findall('\([a-zA-Z0-9\s]*\)',str(typeData))

                                if len(typeData) == 0:
                                    typeData = clipboard.paste()
                                    typeData = typeData[2:-2]
                                    typeData = typeData.split("\r\n")
                                    
                                    for i in range(len(typeData)):
                                        if len(typeData) > 1 and i%2 == 0:
                                            buf = typeData[i] + typeData[i+1]
                                            buf = buf.replace('\r\n','')
                                            threatType.append(buf)
                                            if i+1 == len(typeDate):
                                                break
                                else:
                                    for i in range(len(typeData)):
                                        buf = typeData[i]
                                        buf = buf.replace('(','').replace(')','')
                                        threatType.append(buf)

                            else:
                                for i in range(dataCount):
                                    buf = typeData[0]
                                    buf = buf.replace('\r\n', '')
                                    threatType.append(buf)

            # 유해 도메인 리스트 생성
                        elif addrColText == "유해 도메인 주소":
                            hwp.Run("TableLowerCell")
                            hwp.Run("TableCellBlockExtend")
                            posInfo = hwp.KeyIndicator()
                            curPos = posInfo[8][1:3]
                            prevPos = ''

                            while curPos !=prevPos:
                                hwp.Run("TableLowerCell")
                                prevPos = curPos
                                posInfo = hwp.KeyIndicator()
                                curPos = posInfo[8][1:3]

                            hwp.Run("Copy")
                            urlData = clipboard.paste()
                            urlData = urlData[2:-2]
                            urlData = urlData.split('\r\n')

                            print(index)
                            print(urlData)

                            for i in range(len(urlData)):
                                buf = urlData[i]
                                buf = buf.replace('\r\n', '')
                                threatURL.append(buf)


            # 유해 도메인 탐지명 리스트 생성
                            hwp.Run("Close")
                            hwp.FindCtrl()
                            hwp.Run("ShapeObjTableSelCell")
                            hwp.Run("TableRightCell")
                            hwp.Run("TableLowerCell")
                            hwp.Run("TableCellBlockExtend")

                            posInfo = hwp.KeyIndicator()
                            curPos = posInfo[8][1:3]
                            prevPos = ''

                            while curPos !=prevPos:
                                hwp.Run("TableLowerCell")
                                prevPos = curPos
                                posInfo = hwp.KeyIndicator()
                                curPos = posInfo[8][1:3]

                            hwp.Run("Copy")
                            urlnameData = clipboard.paste()
                            urlnameData = urlnameData[2:-2]
                            urlnameData = urlnameData.split('\r\n')


                            print(index)
                            print(urlnameData)

                            dataCount = len(urlData)

                            if dataCount == len(urlnameData):
                                for i in range(len(urlnameData)):
                                    buf = urlnameData[i]
                                    buf = buf.replace('\r\n', '')
                                    threatUrlName.append(buf)


                            else:
                                for i in range(dataCount):
                                    buf = urlnameData[0]
                                    buf = buf.replace('\r\n', '')
                                    threatUrlName.append(buf)

    # 템플릿이 다른 문서 ㅅㅂ
                    elif colText == "유해 IP주소":
                        hwp.Run("TableLowerCell")
                        hwp.Run("TableCellBlockExtend")
                        posInfo = hwp.KeyIndicator()
                        curPos = posInfo[8][1:3]
                        prevPos = ''

                        while curPos !=prevPos:
                            hwp.Run("TableLowerCell")
                            prevPos = curPos
                            posInfo = hwp.KeyIndicator()
                            curPos = posInfo[8][1:3]

                        hwp.Run("Copy")
                        addrData = clipboard.paste()
                        print(addrData)
                        addrData = addrData[2:-2]
                        addrData = re.findall(r'\b(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\b', str(addrData))
                        
                        print(index)
                        print(addrData)

                        for i in range(len(addrData)):
                            buf = addrData[i]
                            threatAddr.append(buf)
                        
                        for i in range(len(addrData)):
                            buf = '-'
                            threatType.append(buf)
                            

                    else:
                        ctrlCode = ctrlCode.Next

                else:
                    pass
    # 데이터 추출 끝
                print("threatAddr %s %d"%(index, len(threatAddr)))
                print("threatType %d"%len(threatType))
                print("threatURL %d"%len(threatURL))
                print("threatUrlName %d"%len(threatUrlName))
                
                ctrlCode = ctrlCode.Next
        
        except Exception as e:
            now = datetime.datetime.now()
            now = str(now)
            logfOpen = open("error.log","a")
            logfOpen.write("Fail proceed file : {0} {1} {2} \n".format(str(now), str(index), str(e)))
            print("Error from %s %s \n" %(now,index))
            
        finally:
            pass
        hwp.XHwpDocuments.Item(0).Close(isDirty=False)
        os.system('taskkill /f /im Hwp.exe')
        
    print(threatAddr)
    print(threatType)
    print(threatURL)
    print(threatUrlName)
    
    return (threatAddr,threatType,threatURL,threatUrlName)
    
if __name__ == "__main__":
    main()


threatAddr AUtempR 0
threatType 0
threatURL 0
threatUrlName 0
threatAddr AUtempR 0
threatType 0
threatURL 0
threatUrlName 0
[]
[]
[]
[]
